In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from models import CNN_Clasificator, LSTM__Clasificator
from get_embedded_data import get_data_glove_CNN, get_data_glove_LSTM, get_data_tokenizer_MLP, split_data, MAPPING

In [ ]:
device = torch.device("cuda")
device

In [ ]:
# max_len_sentence = 0
# pos = 0
# for i, sentence in enumerate(list_of_words):
#     max_len_sentence, pos = (len(sentence), i) if len(sentence)>max_len_sentence else (max_len_sentence, pos)
# max_len_sentence, pos

In [ ]:
# import nltk
# from nltk.tokenize import word_tokenize
# from gensim.models import Word2Vec
# # nltk.download('punkt')

# import gensim.downloader as api
# corpus = api.load('text8')

# with open('data_set.csv', 'r', encoding='utf-8') as dh:
#     list_of_words = []
#     list_of_targets = []
#     for i, line in enumerate(dh):
#         if i > 0:
#             line = line.strip()
#             line = line.split('@')
#             line[-1] = word_tokenize(line[-1].lower())
#             list_of_words.append(line[-1])
#             list_of_targets.append(line[1])
#     dh.close()

# emb_dim = 100

# model = Word2Vec(corpus, min_count=1)
# model.build_vocab(list_of_words, update=True)
# model.train(list_of_words, total_examples=model.corpus_count, epochs=20)
# model.save('word2vec_100d')

# # model = Word2Vec.load('word2vec/word2vec_100d')

# # Access embeddings
# word_embeddings = model.wv
# print(model.wv.most_similar("folly"))
# print(word_embeddings['superstitious'])


In [ ]:
# word2vec_embeddings = []
# for sentence in list_of_words:
#     emb_sentence = np.empty((100,0))
#     for word in sentence:
#         emb_sentence = np.hstack((emb_sentence, np.reshape(model.wv[word], (100, 1))))
#     word2vec_embeddings.append(torch.from_numpy(emb_sentence))

In [ ]:
# import torchtext
# import torchtext.vocab

# glove = torchtext.vocab.GloVe(name="6B", dim=100)

In [ ]:
# list_of_inc = []
# for i, sen in enumerate(list_of_words):
#     counter = 0
#     for j, wor in enumerate(sen):
#         if wor not in glove:
#             counter += 1
#     if counter >= 1: list_of_inc.append((i, counter))
# print(len(list_of_inc))
# list_of_inc

In [ ]:
# abc = []
# for i, sen in enumerate(list_of_words):
#     for j, wor in enumerate(sen):
#             if wor not in glove: abc.append((list_of_targets[i],i, j, wor))

In [ ]:
# miss_by_aut=[]
# i = 0
# inc = 0
# prev_aut = list_of_inc[0][0]
# for aut , _, _ in list_of_inc:
#     if aut == prev_aut:
#         inc += 1
#     else:
#         miss_by_aut.append([prev_aut, inc])
#         prev_aut = aut
#         i+=1
#         inc = 0
# miss_by_aut.append([aut, inc])
# miss_by_aut

In [ ]:
# import nltk

In [ ]:
# nltk.download()

In [ ]:
X_train, X_test, y_train, y_test = split_data("data_set.csv", "author", "quote", test_size=0.2, separator="@", mapping=MAPPING, labels_to_delete=[])

In [ ]:
# dataloader = get_data_BERT_MLP(20, device)
train_dataloader = get_data_glove_CNN(200, X_train, y_train)
test_dataloader = get_data_glove_CNN(200, X_test, y_test)

In [ ]:
network = CNN_Clasificator().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters())

In [ ]:
max_epoch = 30
network.train()
for epoch in range(max_epoch):

    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = network(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        total += labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    print('[%d/%d] loss: %.3f accuracy: %d' %
          (epoch+1, max_epoch, running_loss / 2000, 100 * correct / total))
    running_loss = 0.0

print('Finished Training')

correct = 0
total = 0
for i, data in enumerate(test_dataloader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = network(inputs)
    total += labels.size(0)
    _, predicted = torch.max(outputs.data, 1)
    correct += (predicted == labels).sum().item()

print(f"Test acc: {100 * correct / total}")

In [ ]:
train_dataloader = get_data_glove_LSTM(200, X_train, y_train)
test_dataloader = get_data_glove_LSTM(200, X_test, y_test)

In [ ]:
network = LSTM__Clasificator(80).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters())

In [ ]:
max_epoch = 30
network.train()
for epoch in range(max_epoch):

    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels, x_len = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        network.prep(200)
        outputs = network(inputs)

        new_preds = torch.empty((0,9)).to(device)
        for i in range(len(outputs)):
            new_preds = torch.vstack((new_preds, outputs[i][x_len[i]-1]))

        loss = criterion(new_preds, labels)
        loss.backward()
        optimizer.step()


        total += labels.size(0)
        _, predicted = torch.max(new_preds.data, 1)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    print('[%d/%d] loss: %.3f accuracy: %d' %
          (epoch+1, max_epoch, running_loss / 2000, 100 * correct / total))
    running_loss = 0.0